# Solução gulosa para o problema de alocação de turmas em salas de aula

## Algoritmo genérico para solução do problema

```
classroom_assign(C = {(Si, Fi) | 0 <= i <= n}):
    ordernar as turmas C em ordem crescente de tempo de início (Si)
    começar com d = 0 salas de aula disponíveis

    para cada turma ci em C:
        se: ci é compatível com uma das d salas abertas, adicione ci a essa sala
        senão: abra a sala d + 1 e adicione ci a essa nova sala
```

Por facilidade na resolução, será considerado apenas a alocação de salas em um dia da semana

In [170]:
def file_reader(path: str):
    with open(path, "r") as file:
        fileLines = file.readlines() 
        return fileLines

In [171]:
import re

def parse_rooms_data(roomsFile: list[str]):
    parsed_rooms = []

    for data in roomsFile:
        parsed_rooms.append(int(data.split('\t')[1].replace('\n', '')))

    return parsed_rooms

def parse_schedules_data(scheduleFile: list[str], weekDay: int):
    all_subjects_per_week_day = []
    parsed_schedules = []

    for i in range(0, len(scheduleFile) - 1):
        subjects_schedules = scheduleFile[i].split('\t')

        if re.match("{:1d}".format(weekDay), subjects_schedules[2]):
            all_subjects_per_week_day.append([
                subjects_schedules[0], 
                int(subjects_schedules[2].replace('\n', '')) % 100
            ])

    current_subject = all_subjects_per_week_day[0][0]
    current_subject_schedule_depth = 0

    for i in range(0, len(all_subjects_per_week_day) - 1):
        if current_subject == all_subjects_per_week_day[i + 1][0]:
            current_subject_schedule_depth += 1
        else:
            parsed_schedules.append([all_subjects_per_week_day[i][0], all_subjects_per_week_day[i][1] - current_subject_schedule_depth, current_subject_schedule_depth])
            current_subject_schedule_depth = 0

        i += 1
        current_subject = all_subjects_per_week_day[i][0]

    return parsed_schedules

In [176]:
def classroom_assign(classes: list, agenda: list[list], total_classrooms: int):
    # ordernar as turmas C em ordem crescente de tempo de início (Si)
    classes.sort(key = lambda i: i[1])          

    for i in range(0, len(classes)):
        d = 0
        j = i
        while (j < len(classes)) and (d < total_classrooms):
            start = classes[j][1]
            finish = classes[j][1] + classes[j][2]
            
            # se: ci é compatível com uma das d salas abertas, adicione ci a essa sala
            if (agenda[start][d] == 0) and (agenda[finish][d] == 0):
                for k in range(start, finish + 1):
                    agenda[k][d] = classes[j][0]
                
                j += 1
            # senão: abra a sala d + 1 e adicione ci a essa nova sala
            else:
                # a sala d + 1 está disponível?
                if (d + 1) < total_classrooms:
                    d += 1
                # voltar a verificar na primeira sala
                else:
                    d = 0
                    j += 1

def print_agenda(agenda: list[list]):
    for lines in agenda:
        print(lines)

In [179]:
roomsFile = file_reader("../data/rooms.txt")
schedulesFile = file_reader("../data/schedules.txt")

rooms = parse_rooms_data(roomsFile)                                         # Salas disponíveis
schedules = parse_schedules_data(schedulesFile, 3)                          # Código e horário das disciplinas

total_hours = 14
agenda = [[0] * len(rooms) for i in range(total_hours)]

classroom_assign(schedules, agenda, len(rooms))